<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# YahooFinance - Chat about a stock trend

**Tags:** #yahoo #finance #ai #chat #plugin #python

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook will generate an AI powered chat plugin for YahooFinance. It will use Python to access the YahooFinance API and create a plugin that can be used to answer questions about stocks and financial markets.

**References:**
- [YahooFinance API Documentation](https://developer.yahoo.com/finance/)
- [Python Library for YahooFinance API](https://pypi.org/project/yahoo-finance/)

## Input

### Import libraries

In [ ]:
from naas_drivers import yahoofinance
import os
from datetime import datetime
import naas
import json

### Setup Variables
- `ticker`: This is a string variable that represents the unique symbol assigned to a particular stock for trading purposes.
- `date_from`: This variable represents the starting date for data retrieval. The type and value of this variable can vary depending on the context.
- `date_to`: This variable represents the end date for data retrieval. The type and value of this variable can vary depending on the context.
- `interval`: This is a string variable that represents the frequency of data points in the time series data.
- `moving_averages`: This is a list that contains the periods for which moving averages should be calculated in the time series data.
- `cron`: This is a string variable representing a cron expression, which is used to schedule tasks in Unix-like operating systems. The
- `output_dir`: This variable stores the directory path where the output files will be saved.
- `file_name`: This variable stores the file name in CSV.

In [ ]:
# Inputs
ticker = "TSLA"
date_from = -100
date_to = "today"
interval = "1d"
moving_averages = [20, 50]
cron = "0 8 * * *"

# Outputs
output_dir = f"yahoo/ticker/{ticker}"
file_name = f"{ticker}.csv"

## Model

### Create output file path
The output will be saved in CSV in the output_dir with datestamp

In [ ]:
def create_file_path(output_dir, file_name):
    # Init
    file_path = ""
    
    # Create output directories
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    # Create file path with datestamp
    today = datetime.now().strftime("%Y%m%d")
    file_path = f"{output_dir}/{today}_{file_name}"
    return file_path

file_path = create_file_path(output_dir, file_name)
print("💾 File will be saved in:", file_path)

### Get dataset from Yahoo Finance

In [ ]:
df = yahoofinance.get(
    ticker,
    date_from=date_from,
    date_to=date_to,
    interval=interval,
    moving_averages=moving_averages,
)
df

### Create prompt for plugin
We used Playground to refined it: https://platform.openai.com/playground?mode=chat&model=gpt-4

In [ ]:
prompt = f"""
You are Abi trader in the NY Stock Exchange. You want to analyze the stock of {ticker} and identity when I need to buy or sell, what's the currents stock trend and
when did the 20MA crossed the 50MA?

DataFrame Structure:
- Number of rows: {df.shape[0]}
- Number of columns: {df.shape[1]}

Column Names:
1. Date: This column represents the date corresponding to the ticker data. It typically includes daily timestamps or dates on which the data was recorded.
2. Open: The "Open" column indicates the opening price of the stock or security on a particular trading day. It represents the price at which the market first started trading the security during that day.
3. High: The "High" column represents the highest price reached by the stock or security during the trading day.
4. Low: The "Low" column indicates the lowest price reached by the stock or security during the trading day.
5. Close: The "Close" column represents the closing price of the stock or security on a particular trading day. It denotes the final price at which the market traded the security before the day's trading session ended.
6. Adj Close: The "Adj Close" column refers to the adjusted closing price of the stock or security. It takes into account factors such as dividends, stock splits, or other corporate actions that may affect the stock price.
7. Volume: The "Volume" column represents the total number of shares or units of the security that were traded during a specific trading day.
8. Ticker: The "Ticker" column identifies the ticker symbol or code assigned to a specific stock or security. It serves as a unique identifier for the financial instrument being traded.
9. MA20: The "MA20" column refers to the Moving Average (MA) over a 20-day period. It is a commonly used technical indicator that calculates the average price of the security over the specified time frame, in this case, 20 days.
10. MA50: The "MA50" column represents the Moving Average (MA) over a 50-day period. Similar to the MA20, it calculates the average price of the security over a longer time frame of 50 days.

Sample Data:
{df.to_string(index=False)}
"""
prompt

### Create MyChatGPT plugin
- `name`: It represents the name of the plugin, which in this case is "LinkedIn chat posts".
- `model`: It specifies the model to be used by the plugin, and in this case, it is "gpt-4".
- `temperature`: It determines the creativity level of the generated content, with higher values resulting in more diverse outputs. In this case, the temperature is set to 1.
- `max_tokens`: It sets the maximum number of tokens allowed in the generated content. In this case, the limit is set to 2084 tokens.
- `prompt`: Prompt used in your plugin

In [ ]:
def create_plugin(
    name,
    model,
    temperature,
    max_tokens,
    prompt,
    output_dir
):
    # Init
    file_path = ""
    plugin_name = "linkedin_posts_analyzer.json"
    
    # Create output directories
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    # Create file path
    file_path = f"{output_dir}/{plugin_name}"
        
    # Create json
    plugin = {
        "name": name,
        "model": model,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "prompt": prompt,
    }

    # Save json to file
    with open(file_path, "w") as f:
        json.dump(plugin, f)
    return file_path

name = "Yahoo Abi trader"
model = "gpt-4"
temperature = 1
max_tokens = 2084
plugin_path = create_plugin(
    name,
    model,
    temperature,
    max_tokens,
    prompt,
    output_dir,
)
print("✅ Plugin path:", plugin_path)

## Output

### Save YahooFinance data

In [ ]:
df.to_csv(file_path, index=False)

### Save plugin in naas asset
You can now use in your MyChatGPT by copy/pasting the URL after the command `/use `

In [ ]:
naas.asset.add(plugin_path, params={"inline": True})

### Add Naas scheduler
Schedule your notebook with the naas scheduler feature

In [ ]:
naas.scheduler.add(cron=cron)

# to de-schedule this notebook, simply run the following command:
# naas.scheduler.delete()